In [1]:
# Necessary Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score
import optuna
import logging

optuna.logging.set_verbosity(optuna.logging.WARNING)


In [2]:
# Exploratory Data Analysis
# Load Data
women_train = pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/training.csv')
women_test = pd.read_csv("/kaggle/input/widsdatathon2024-challenge1/test.csv")
sample_submission = pd.read_csv("/kaggle/input/widsdatathon2024-challenge1/sample_submission.csv")


In [3]:
# Feature Engineering
# Drop unnecessary columns for modeling
drop_columns = ['metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'bmi','patient_race']
women_train = women_train.drop(columns=drop_columns)
women_test = women_test.drop(columns=drop_columns)

In [4]:
# Text preprocessing for 'breast_cancer_diagnosis_desc'
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation and special characters
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    return text

women_train['breast_cancer_diagnosis_desc'] = women_train['breast_cancer_diagnosis_desc'].apply(preprocess_text)
women_test['breast_cancer_diagnosis_desc'] = women_test['breast_cancer_diagnosis_desc'].apply(preprocess_text)


In [5]:
# Define features and target variable
X = women_train.drop(columns=['DiagPeriodL90D'])
y = women_train['DiagPeriodL90D']
# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:

# Preprocessing for numerical and categorical data
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])



In [7]:
# Model Building and Evaluation
# Define XGBoost model with Optuna hyperparameter optimization
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'random_state': 42
    }

    xgb_model = XGBClassifier(**params)

    clf = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', xgb_model)])
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_pred)

    return auc


In [8]:
# Define CatBoost model with Optuna hyperparameter optimization
def objective_catboost(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0),
        'random_state': 42,
        'verbose': 0
    }

    catboost_model = CatBoostClassifier(**params)

    clf = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', catboost_model)])
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_pred)

    return auc


In [9]:
# Create and run the Optuna study for XGBoost
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=30)

# Get the best parameters for XGBoost
best_params_xgb = study_xgb.best_params

In [10]:
# Train XGBoost model with best hyperparameters
xgb_model = XGBClassifier(**best_params_xgb, random_state=42)
clf_xgb = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', xgb_model)])
clf_xgb.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['patient_id', 'patient_zip3', 'patient_age', 'population', 'density',
       'age_median', 'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s',
       'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'male',
       'female',...
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.011003661188363923, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=9,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=264,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=42, ...))])

In [11]:
# Evaluate XGBoost model
y_pred_xgb = clf_xgb.predict_proba(X_valid)[:, 1]
auc_xgb = roc_auc_score(y_valid, y_pred_xgb)
print(f'XGBoost: AUC = {auc_xgb:.4f}')


XGBoost: AUC = 0.8038


In [12]:
# Create and run the Optuna study for CatBoost
study_catboost = optuna.create_study(direction='maximize')
study_catboost.optimize(objective_catboost, n_trials=30)

# Get the best parameters for CatBoost
best_params_catboost = study_catboost.best_params


In [13]:
# Train CatBoost model with best hyperparameters
catboost_model = CatBoostClassifier(**best_params_catboost, random_state=42, verbose=0)
clf_catboost = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', catboost_model)])
clf_catboost.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['patient_id', 'patient_zip3', 'patient_age', 'population', 'density',
       'age_median', 'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s',
       'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'male',
       'female',...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['payer_type', 'patient_state', 'patient_gender',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'],
      dtype='object'))])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x7a6567ed5e40>)])

In [14]:
# Evaluate CatBoost model
y_pred_catboost = clf_catboost.predict_proba(X_valid)[:, 1]
auc_catboost = roc_auc_score(y_valid, y_pred_catboost)
print(f'CatBoost: AUC = {auc_catboost:.4f}')



CatBoost: AUC = 0.8001


In [15]:
# Ensemble XGBoost and CatBoost models
voting_clf = VotingClassifier([('xgb', clf_xgb), ('catboost', clf_catboost)], voting='soft')



In [16]:
# Perform Multi-Level Cross-Validation for the ensemble model
num_splits = 5  # Number of Group K-Fold splits
auc_scores_ensemble = []

gkf = GroupKFold(n_splits=num_splits)
for train_index, valid_index in gkf.split(X, y, groups=women_train['patient_zip3']):
    X_train_gkf, X_valid_gkf = X.iloc[train_index], X.iloc[valid_index]
    y_train_gkf, y_valid_gkf = y.iloc[train_index], y.iloc[valid_index]
    
    voting_clf.fit(X_train_gkf, y_train_gkf)
    y_pred_ensemble_gkf = voting_clf.predict_proba(X_valid_gkf)[:, 1]
    auc_ensemble_gkf = roc_auc_score(y_valid_gkf, y_pred_ensemble_gkf)
    auc_scores_ensemble.append(auc_ensemble_gkf)

print("Ensemble AUC Scores (Multi-Level Cross-Validation):", auc_scores_ensemble)



Ensemble AUC Scores (Multi-Level Cross-Validation): [0.8069485616332422, 0.79716824250487, 0.811265829317541, 0.7938207625707625, 0.7812504793606478]


In [17]:
# Make predictions on test set with the ensemble model
final_predictions_ensemble = []

for _ in range(num_splits):
    voting_clf.fit(X, y)  # Fit on the entire dataset
    final_predictions_ensemble.append(voting_clf.predict_proba(women_test)[:, 1])

# Average predictions across Multi-Level Cross-Validation splits
final_predictions_ensemble = np.mean(final_predictions_ensemble, axis=0)


In [18]:
# Prepare submission file
submission_df = pd.DataFrame({'patient_id': women_test['patient_id'], 'DiagPeriodL90D': final_predictions_ensemble})
submission_df.to_csv('submission.csv', index=False)